# Measuring uptake of inhaled budesonide prescription guidance following COVID-19 infection

## Setup, data loading and prepreprocessing

In [2]:
import pandas as pd

In [36]:
## low count suppression function
def suppress(df_to_suppress,columns,n=10):
    for c in columns:
        if not c in df_to_suppress.columns:
            raise ValueError(f'column {c} not in dataframe')

    df = df_to_suppress
    for c in df.columns[df.columns.isin(columns)]:
        df[[c]] = df[[c]].astype(str)
        df[[c]] = df[c].apply(lambda x: '<10' if (int(x)>0 and int(x)<n ) else x)
    return df

In [5]:
df = pd.read_csv('../output/input.csv')

In [6]:
#exclude LFT tests, include unknowns
df.first_positive_test_type= df.first_positive_test_type.fillna('Unknown')
df = df[df.first_positive_test_type!="LFT"]

In [21]:
#add cohort label [over 65s, over 55s with sheilding flag, over 55s with non-sheilding flag]
df['cohort'] = df.apply(lambda x: x.age_band if x.age_band == '65_plus' else str(x.age_band) + ('H' if x.primis_shield==1 else 'L'),axis=1)


## COVID-19 Test Types

In [7]:
df_test_counts = df.groupby('first_positive_test_type').count()['patient_id'].reset_index().rename(columns={'patient_id':'Patient count',"first_positive_test_type":"Type of first recorded positive test"})
supressed_test_type_counts = suppress(df_test_counts,['Patient count'])
supressed_test_type_counts.style.hide_index()

## Budesonide prescriptions by cohort

In [51]:
supressed_bd_prescriptions = suppress(df.groupby('budesonide_prescription').count()['patient_id'].reset_index().rename(columns={"budesonide_prescription":"Count of budesonide prescriptions","patient_id":"Count of patients"}),["Count of patients"])
supressed_bd_prescriptions.style.hide_index()

Count of budesonide prescriptions,Count of patients
-1,12
0,725
1,185
2,68
3,<10
4,<10


In [52]:
supressed_bd_prescriptions_by_cohort = suppress(df.groupby(['cohort','budesonide_prescription']).count()['patient_id'].reset_index().rename(columns={"budesonide_prescription":"Count of budesonide prescriptions","patient_id":"Count of patients"}),['Count of patients'])
supressed_bd_prescriptions_by_cohort.style.hide_index()

cohort,Count of budesonide prescriptions,Count of patients
55_65L,0,31
55_65L,1,<10
55_65L,2,<10
65_plus,0,33
65_plus,1,<10
lt_55H,0,28
lt_55H,1,<10
lt_55H,2,<10
lt_55L,-1,<10
lt_55L,0,278
